In [ ]:
//@version=6
indicator("Asset Rotation System [InvestorUnknown] — ChartSymbol + USDC.D", max_bars_back=5000)
import TradingView/ta/9

//──────────────────────────────
// FUNCTIONS
//──────────────────────────────
f_remove_exchange_name(name) =>
    parts = str.split(name, ":")
    result = array.size(parts) > 1 ? array.get(parts, 1) : name
    result

f_trend_col(x) =>
    x > 0 ? color.green : color.red

f_rsi(src, len) =>
    rsi = ta.rsi(src, len)
    rsi > 50 ? 1 : 0

f_cci(src, len) =>
    cci = ta.cci(src, len)
    cci > 0 ? 1 : 0

f_supertrend(factor, len) =>
    [st, dir] = ta.supertrend(factor, len)
    -dir > 0 ? 1 : 0

f_dmi(len) =>
    [dip, dim, adx] = ta.dmi(len, 1)
    dip > dim ? 1 : 0

f_psar(start, increment, max) =>
    sar = ta.sar(start, increment, max)
    sar < close ? 1 : 0

f_all(rsi_src, rsi_len, cci_src, cci_len, st_factor, st_len, dmi_len, sar_start, sar_inc, sar_max) =>
    r_val = f_rsi(rsi_src, rsi_len)
    c_val = f_cci(cci_src, cci_len)
    st_val = f_supertrend(st_factor, st_len)
    d_val = f_dmi(dmi_len)
    psar_val = f_psar(sar_start, sar_inc, sar_max)
    (r_val + c_val + st_val + d_val + psar_val) > 0 ? 1 : 0

f_oppose(x) =>
    x == 1 ? 0 : (x == 0 ? 1 : 0)

f_CM_RSIStoch(src, rsi_len, stoch_len, stoch_k, rsi_lower_val, stoch_lower_val) =>
    rsi_val     = ta.rsi(src, rsi_len)
    stoch_val   = ta.stoch(high, low, src, stoch_len)
    stoch_k_val = ta.sma(stoch_val, stoch_k)
    trend_up = rsi_val > 50 and stoch_k_val > 50
    trend_up ? 1 : 0

f_trend(clas, rsi_src, rsi_len, cci_src, cci_len, st_factor, st_len, dmi_len, sar_start, sar_inc, sar_max, stoch_len, stoch_k, rsi_lower_val, stoch_lower_val) =>
    sig = switch clas
        "RSI"         => f_rsi(rsi_src, rsi_len)
        "CCI"         => f_cci(cci_src, cci_len)
        "SuperTrend"  => f_supertrend(st_factor, st_len)
        "DMI"         => f_dmi(dmi_len)
        "PSAR"        => f_psar(sar_start, sar_inc, sar_max)
        "CM_RSIStoch" => f_CM_RSIStoch(rsi_src, rsi_len, stoch_len, stoch_k, rsi_lower_val, stoch_lower_val)
        "All"         => f_all(rsi_src, rsi_len, cci_src, cci_len, st_factor, st_len, dmi_len, sar_start, sar_inc, sar_max)
    [sig, f_oppose(sig)]

f_age() =>
    bars = bar_index + 1
    math.min(bars, 5000)

f_roc() =>
    roc = ta.roc(close, 1)
    roc / 100

f_since(active) =>
    var count = 0
    if active
        count += 1
    count > 2 ? count - 1 : count

f_BestPerformingAsset(ticker1, v1, ticker2, v2, ticker3, v3, ticker4, v4, ticker5, v5, ticker6, v6, ticker7, v7, ticker8, v8, ticker9, v9, ticker10, v10) =>
    var tickers = array.new_string(10, "")
    var values  = array.new_float(10, 0.0)
    array.set(tickers, 0, ticker1), array.set(values, 0, v1)
    array.set(tickers, 1, ticker2), array.set(values, 1, v2)
    array.set(tickers, 2, ticker3), array.set(values, 2, v3)
    array.set(tickers, 3, ticker4), array.set(values, 3, v4)
    array.set(tickers, 4, ticker5), array.set(values, 4, v5)
    array.set(tickers, 5, ticker6), array.set(values, 5, v6)
    array.set(tickers, 6, ticker7), array.set(values, 6, v7)
    array.set(tickers, 7, ticker8), array.set(values, 7, v8)
    array.set(tickers, 8, ticker9), array.set(values, 8, v9)
    array.set(tickers, 9, ticker10), array.set(values, 9, v10)
    var highestIndex = 0
    for i = 0 to 9
        if array.get(values, i) > array.get(values, highestIndex)
            highestIndex := i
    array.get(tickers, highestIndex)

f_equity(t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, best_asset, r1, r2, r3, r4, r5, r6, r7, r8, r9, r10, backtest) =>
    var float equity = na
    var float peak_equity = na
    var float max_drawdown = 0.0
    if backtest
        a = switch best_asset[1]
            t1  => r1
            t2  => r2
            t3  => r3
            t4  => r4
            t5  => r5
            t6  => r6
            t7  => r7
            t8  => r8
            t9  => r9
            t10 => r10
            "USD" => 0
        equity := na(equity[1]) ? 1 : equity[1] * (1 + a)
        peak_equity := math.max(na(peak_equity) ? equity : peak_equity, equity)
        current_drawdown = peak_equity > 0 ? (peak_equity - equity) / peak_equity : 0
        if current_drawdown > max_drawdown
            max_drawdown := current_drawdown
        if equity > peak_equity
            peak_equity := equity
    else
        equity := 1
    [equity, math.round(max_drawdown, 4)]

f_buy_and_ho_
